In [49]:
%%capture --no-stderr
%pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters javalang

In [50]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

In [51]:
 # [ File Ingestion Node ] --> [ Code Parsing Node ] --> [ Rubric Mapping Node ] 
                             # --> [ Code Comparison Node ] --> [ Test Case Generation Node ] 
                              #    --> [ Error Detection Node ] --> [ Feedback Generation Node ] 
                                 #     --> [ Scoring Node ] --> [ Result Output Node ]

In [57]:
def read_file_content(file_path):
    with open(file_path, 'r') as file:
        return file.read()

student_code = read_file_content('student_solution.md')
model_solution = read_file_content('model_solution.md')
question = read_file_content('question.md')
rubric = read_file_content('rubric.md')

print(student_code)


```java
import java.util.Scanner;

public class StringManipulator {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        System.out.print("Enter a string: ");
        String input = sc.next();

        System.out.println("Original String: " + input);
        System.out.println("Uppercase String: " + input.toLowerCase());

        String reversed = "";
        for (int i = 0; i <= input.length(); i++) { 
            reversed += input.charAt(i); 
        }
        System.out.println("Reversed String: " + reversed);

        System.out.println("Number of Characters: " + (input.length() - 1)); 
    }
}
```



In [56]:
def evaluate_code(java_code):
    
    errors = []
    
    # Check for class definition
    if 'public class StringManipulator' not in java_code:
        errors.append('Missing or incorrect class definition. Should be "public class StringManipulator".')
    
    # Check for main method
    if 'public static void main' not in java_code:
        errors.append('Missing main method.')
    
    # Check for Scanner usage
    if 'Scanner' not in java_code:
        errors.append('Missing Scanner object for user input.')
    
    # Check for required string operations
    required_methods = ['toUpperCase()', 'reverse()', 'length()']
    for method in required_methods:
        if method not in java_code:
            errors.append(f'Missing string operation: {method}')
    
    # Check for line count constraint
    code_lines = java_code.strip().split('\n')
    if len([line for line in code_lines if line.strip() and not line.strip().startswith('//')]) > 20:
        errors.append('Code exceeds the 20-line limit.')
    
    if not errors:
        return {'status': 'Passed', 'comments': 'No issues found.'}
    else:
        return {'status': 'Failed', 'comments': errors}

In [62]:
def load_student_solution(file_path):
    
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def parse_java_code(md_content):
   
    pattern = re.compile(r'```java\s+([\s\S]*?)```', re.MULTILINE)
    matches = pattern.findall(md_content)
    java_codes = [match.strip() for match in matches]
    return java_codes

# Example usage
file_path = 'student_solution.md'
md_content = load_student_solution(file_path)
java_codes = parse_java_code(md_content)

for idx, code in enumerate(java_codes, start=1):
    print(f"--- Java Code Block {idx} ---\n{code}\n")

--- Java Code Block 1 ---
import java.util.Scanner;

public class StringManipulator {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        System.out.print("Enter a string: ");
        String input = sc.next();

        System.out.println("Original String: " + input);
        System.out.println("Uppercase String: " + input.toLowerCase());

        String reversed = "";
        for (int i = 0; i <= input.length(); i++) { 
            reversed += input.charAt(i); 
        }
        System.out.println("Reversed String: " + reversed);

        System.out.println("Number of Characters: " + (input.length() - 1)); 
    }
}



In [81]:
import re

def extract_classes(java_code):
    """Extract individual Java classes from the code."""
    class_pattern = re.compile(r'\bpublic class\s+(\w+)\s*{([^}]*)}', re.DOTALL)
    classes = class_pattern.findall(java_code)
    return {class_name: class_body.strip() for class_name, class_body in classes}

def extract_rubric_criteria(rubric_text):
    """Extract rubric criteria from the rubric text."""
    criteria = re.findall(r'(\d+\.\d+)\s*-\s*(.*)', rubric_text)
    return {float(crit[0]): crit[1].strip() for crit in criteria}

def map_classes_to_rubric(student_code, model_code, rubric):
    """Map individual Java classes from both model and student code to relevant rubric criteria."""
    # Extract classes from both student and model code
    student_classes = extract_classes(student_code)
    model_classes = extract_classes(model_code)

    # Extract rubric criteria
    rubric_criteria = extract_rubric_criteria(rubric)

    # Create a mapping of classes to relevant rubric criteria
    class_rubric_mapping = {}
    
    for class_name in student_classes.keys():
        if class_name in model_classes:
            # Assume both classes exist in the student and model codes
            # For demonstration, we consider the same rubric criteria for simplicity
            relevant_criteria = []
            for score, criterion in rubric_criteria.items():
                # This logic should ideally match the content of the classes to the criteria.
                relevant_criteria.append((score, criterion))

            class_rubric_mapping[class_name] = {
                'student': relevant_criteria,
                'model': relevant_criteria
            }

    return class_rubric_mapping

# Example usage
rubric = """
10.0 - Excellent
8.0 - Good
6.0 - Fair
4.0 - Needs Improvement
2.0 - Poor
"""
student_code = """
public class StringManipulator {
    public static void main(String[] args) {
        // Student code here
    }
}

public class AnotherClass {
    // Student code here
}
"""

model_code = """
public class StringManipulator {
    public static void main(String[] args) {
        // Model code here
    }
}

public class AnotherClass {
    // Model code here
}
"""

# Mapping classes to rubric criteria
class_rubric = map_classes_to_rubric(student_code, model_code, rubric)

# Print the mapping
for class_name, criteria in class_rubric.items():
    print(f"Class: {class_name}")
    print("  Student Criteria:")
    for score, criterion in criteria['student']:
        print(f"    - Score: {score}, Criterion: {criterion}")

    print("  Model Criteria:")
    for score, criterion in criteria['model']:
        print(f"    - Score: {score}, Criterion: {criterion}")


Class: StringManipulator
  Student Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
  Model Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
Class: AnotherClass
  Student Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
  Model Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor


In [76]:
def compare_code(student_code_ast, model_solution_ast):
    if str(student_code_ast) == str(model_solution_ast):
        return "Code matches model solution"
    else:
        return "Code differs from model solution"

# Example Usage
model_solution_ast = parse_java_code(model_solution)
comparison_result = compare_code(student_code_ast, model_solution_ast)
print(comparison_result)


Code matches model solution


In [83]:
def detect_errors(student_code_ast):
    errors = []
    
    if 'for' in str(student_code_ast):
        if 'input.length()' in str(student_code_ast) and 'i <= input.length()' in str(student_code_ast):
            errors.append(" error  ")
    
    return errors

errors_detected = detect_errors(student_code_ast)
print(errors_detected)


[]


In [70]:
def generate_feedback(errors, rubric_criteria):
    feedback = []
    
    if errors:
        for error in errors:
            feedback.append(f"Issue: {error}. Suggested correction: Check loop conditions.")
    else:
        feedback.append("No errors found. Code is correct.")
    
    return feedback

feedback = generate_feedback(errors_detected, rubric_criteria)
print("\n".join(feedback))


No errors found. Code is correct.


In [71]:
def score_code(errors_detected, rubric_criteria):
    total_score = 10  
    for error in errors_detected:
        total_score -= 1  
    return max(total_score, 0)  

final_score = score_code(errors_detected, rubric_criteria)
print(f"Final Score: {final_score}/10")


Final Score: 10/10


In [72]:
def output_result(score, feedback):
    print(f"Final Score: {score}/10")
    print("Feedback:")
    print("\n".join(feedback))

output_result(final_score, feedback)


Final Score: 10/10
Feedback:
No errors found. Code is correct.


In [86]:
import re

def extract_classes(java_code):
    """Extract individual Java classes from the code."""
    class_pattern = re.compile(r'\bpublic class\s+(\w+)\s*{([^}]*)}', re.DOTALL)
    classes = class_pattern.findall(java_code)
    return {class_name: class_body.strip() for class_name, class_body in classes}

def extract_rubric_criteria(rubric_text):
    """Extract rubric criteria from the rubric text."""
    criteria = re.findall(r'(\d+\.\d+)\s*-\s*(.*)', rubric_text)
    return {float(crit[0]): crit[1].strip() for crit in criteria}

def map_classes_to_rubric(student_code, model_code, rubric):
    """Map individual Java classes from both model and student code to relevant rubric criteria."""
    # Extract classes from both student and model code
    student_classes = extract_classes(student_code)
    model_classes = extract_classes(model_code)

    # Extract rubric criteria
    rubric_criteria = extract_rubric_criteria(rubric)

    # Create a mapping of classes to relevant rubric criteria
    class_rubric_mapping = {}
    
    for class_name in student_classes.keys():
        if class_name in model_classes:
            # Assume both classes exist in the student and model codes
            # For demonstration, we consider the same rubric criteria for simplicity
            relevant_criteria = []
            for score, criterion in rubric_criteria.items():
                # This logic should ideally match the content of the classes to the criteria.
                relevant_criteria.append((score, criterion))

            class_rubric_mapping[class_name] = {
                'student': relevant_criteria,
                'model': relevant_criteria
            }

    return class_rubric_mapping

def evaluate_marks(class_rubric_mapping):
    """Evaluate marks for each class based on model criteria and calculate total marks."""
    total_marks = 0
    evaluation_results = {}
    
    for class_name, criteria in class_rubric_mapping.items():
        student_scores = []
        
        print(f"Evaluating Class: {class_name}")
        
        # Score the student code against model criteria
        for score, criterion in criteria['student']:
            # Placeholder for actual evaluation logic
            # This is where you would implement checks against the model criteria.
            # For this demonstration, we'll assign full marks for simplicity.
            student_scores.append(score)
            print(f"  - Criterion: {criterion}, Score: {score}")
        
        # Calculate total for this class
        class_total = sum(student_scores)
        total_marks += class_total
        evaluation_results[class_name] = class_total
    
    return evaluation_results, total_marks

# Example usage
rubric = """
10.0 - Excellent
8.0 - Good
6.0 - Fair
4.0 - Needs Improvement
2.0 - Poor
"""
student_code = """
public class StringManipulator {
    public static void main(String[] args) {
        // Student code here
    }
}

public class AnotherClass {
    // Student code here
}
"""

model_code = """
public class StringManipulator {
    public static void main(String[] args) {
        // Model code here
    }
}

public class AnotherClass {
    // Model code here
}
"""

# Mapping classes to rubric criteria
class_rubric = map_classes_to_rubric(student_code, model_code, rubric)

# Print the mapping
for class_name, criteria in class_rubric.items():
    print(f"Class: {class_name}")
    print("  Student Criteria:")
    for score, criterion in criteria['student']:
        print(f"    - Score: {score}, Criterion: {criterion}")

    print("  Model Criteria:")
    for score, criterion in criteria['model']:
        print(f"    - Score: {score}, Criterion: {criterion}")

# Evaluate marks for each class and calculate total marks
evaluation_results, total_marks = evaluate_marks(class_rubric)

# Print evaluation results
print("\nEvaluation Results:")
for class_name, class_total in evaluation_results.items():
    print(f"Class: {class_name}, Total Marks: {class_total}")

print(f"Total Marks across all classes: {total_marks}")


Class: StringManipulator
  Student Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
  Model Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
Class: AnotherClass
  Student Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
  Model Criteria:
    - Score: 10.0, Criterion: Excellent
    - Score: 8.0, Criterion: Good
    - Score: 6.0, Criterion: Fair
    - Score: 4.0, Criterion: Needs Improvement
    - Score: 2.0, Criterion: Poor
Evaluating Class: StringManipulator
  - Criterion: Excellent, Score: 10.0
  - Criterion: Good, Score: 8.0
  - Criterion: Fa